## Building Tilesets using Xarray-Spatial and Datashader

Xarray-Spatial provides `render_tiles` which is a utility function for creating tilesets.

In [ ]:
import pandas as pd
import datashader as ds
import numpy as np

from xrspatial.tiles import render_tiles

In [ ]:
import geopandas as gpd

In [ ]:
world = gpd.read_file(
    gpd.datasets.get_path('naturalearth_lowres')
)
world = world.to_crs("EPSG:3857") 
world = world[world.continent != 'Antarctica']
world.plot(figsize=(7, 7))

## Define Tiling Component Functions

### Create `load_data_func`
- accepts `x_range` and `y_range` arguments which correspond to the ranges of the supertile being rendered.
- returns a dataframe-like object (pd.Dataframe / dask.Dataframe)
- this example `load_data_func` creates a pandas dataframe with `x` and `y` fields sampled from a wald distribution 

In [ ]:
import pandas as pd
import numpy as np

def load_data_func(x_range, y_range):
    return world.cx[y_range[0]:y_range[1], x_range[0]:x_range[1]]

### Create `rasterize_func`
- accepts `df`, `x_range`, `y_range`, `height`, `width` arguments which correspond to the data, ranges, and plot dimensions of the supertile being rendered.
- returns an `xr.DataArray` object representing the aggregate.

In [ ]:
import datashader as ds
from spatialpandas import GeoDataFrame

def rasterize_func(df, x_range, y_range, height, width):
    spatialpandas_df = GeoDataFrame(df, geometry='geometry')
    # aggregate
    cvs = ds.Canvas(x_range=x_range, y_range=y_range,
                    plot_height=height, plot_width=width)
    agg = cvs.polygons(spatialpandas_df, 'geometry')
    return agg

### Create `shader_func`
- accepts `agg (xr.DataArray)`, `span (tuple(min, max))`.  The span argument can be used to control color mapping / auto-ranging across supertiles.
- returns an `ds.Image` object representing the shaded image.

In [ ]:
import datashader.transfer_functions as tf
from datashader.colors import viridis

def shader_func(agg, span=None):
    img = tf.shade(agg, cmap=['black', 'teal'], span=span, how='log')
    img = tf.set_background(img, 'black')
    return img

### Create `post_render_func`
- accepts `img `, `extras` arguments which correspond to the output PIL.Image before it is write to disk (or S3), and addtional image properties.
- returns image `(PIL.Image)`
- this is a good place to run any non-datashader-specific logic on each output tile.

In [ ]:
def post_render_func(img, **kwargs):
    info = "x={},y={},z={}".format(kwargs['x'], kwargs['y'], kwargs['z'])
    return img

## Render tiles to local filesystem

In [ ]:
full_extent_of_data = (-20e6, 20e6,
                       -20e6, 20e6)

output_path = '/Users/bcollins/temp/test_world/'
results = render_tiles(full_extent_of_data,
                       range(0, 8),
                       load_data_func=load_data_func,
                       rasterize_func=rasterize_func,
                       shader_func=shader_func,
                       post_render_func=post_render_func,
                       output_path=output_path,
                       color_ranging_strategy=(0,2))

### Preview the tileset using Bokeh
- Browse to the tile output directory and start an http server:

```bash
$> cd test_tiles_output
$> python -m http.server

Starting up http-server, serving ./
Available on:
  http://127.0.0.1:8080
  http://192.168.1.7:8080
Hit CTRL-C to stop the server
```

- build a `bokeh.plotting.Figure`

In [ ]:
from bokeh.io import output_notebook, show, output_file
output_notebook()

### Preview Tiles

In [ ]:
xmin, ymin, xmax, ymax = full_extent_of_data
from bokeh.plotting import figure
from bokeh.models.tiles import WMTSTileSource

p = figure(width=800, height=800, 
           x_range=(int(-20e6), int(20e6)),
           y_range=(int(-20e6), int(20e6)),
           tools="pan,wheel_zoom,reset")
p.axis.visible = False
p.background_fill_color = 'black'
p.grid.grid_line_alpha = 0
p.add_tile(WMTSTileSource(url="http://localhost:10000/{Z}/{X}/{Y}.png"),
           render_parents=False)
show(p)